In [1]:
!apt install espeak-ng sox




The following additional packages will be installed:
  espeak-ng-data libespeak-ng1 libopencore-amrnb0 libopencore-amrwb0
  libpcaudio0 libsonic0 libsox-fmt-alsa libsox-fmt-base libsox3 libwavpack1
Suggested packages:
  libsox-fmt-all
The following NEW packages will be installed:
  espeak-ng espeak-ng-data libespeak-ng1 libopencore-amrnb0 libopencore-amrwb0
  libpcaudio0 libsonic0 libsox-fmt-alsa libsox-fmt-base libsox3 libwavpack1
  sox
0 upgraded, 12 newly installed, 0 to remove and 98 not upgraded.
Need to get 5,142 kB of archives.
After this operation, 13.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 libpcaudio0 amd64 1.1-6build2 [8,956 B]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 libsonic0 amd64 0.2.0-11build1 [10.3 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 espeak-ng-data amd64 1.50+dfsg-10ubuntu0.1 [3,956 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 libespeak-ng

In [2]:
%pip install pocketsphinx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.2/29.2 MB 10.5 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [3]:
from pydub import AudioSegment


/usr/local/lib/python3.12/dist-packages/pydub/utils.py:300: SyntaxWarning: invalid escape sequence '\('
  m = re.match('([su]([0-9]{1,2})p?) \(([0-9]{1,2}) bit\)$', token)
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:301: SyntaxWarning: invalid escape sequence '\('
  m2 = re.match('([su]([0-9]{1,2})p?)( \(default\))?$', token)
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:310: SyntaxWarning: invalid escape sequence '\('
  elif re.match('(flt)p?( \(default\))?$', token):
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:314: SyntaxWarning: invalid escape sequence '\('
  elif re.match('(dbl)p?( \(default\))?$', token):


In [4]:
ENGLISH_MAPPING = """
ɑː AA
æ AE
ə AH
ɐ AH
ʌ AH
ɔː AO
aʊ AW
aɪ AY
b B
tʃ CH
d D
ð DH
ɛ EH
ɚɹ ER
ɚ ER
ɜː ER
eɪ EY
f F
ɡ G
h HH
ɪ IH
i IY
iː IY
dʒ JH
k K
l L
m M
n N
ŋ NG
oʊ OW
ɔɪ OY
p P
ɹ R
s S
ʃ SH
t T
θ TH
ʊ UH
uː UW
v V
w W
j Y
z Z
ʒ ZH
ɾ D
"""

In [5]:
def get_mapping(MAPPING):
    espeak_to_cmudict = {}
    for line in MAPPING.split("\n"):
        if line == "":
            continue
        line = line.strip()
        parts = line.split(" ")
    
        if len(parts) != 2:
            print(line)
            continue
        k, v = line.split(" ")
        if not k in espeak_to_cmudict:
            espeak_to_cmudict[k] = v
    return espeak_to_cmudict

In [6]:
espeak_to_cmudict = get_mapping(ENGLISH_MAPPING)

In [7]:
import re

cmudict_keys = espeak_to_cmudict.keys()
cmudict_keys = sorted(cmudict_keys, key=len, reverse=True)
espeak_regex = re.compile(rf"({'|'.join(cmudict_keys)})")

def cmudictify(espeak):
    espeak = espeak.replace("ˈ", "").replace("ˌ", "")
    return " ".join([espeak_to_cmudict[x] for x in re.findall(espeak_regex, espeak)])

In [8]:
def normword(text):
    text = text.strip(",.;:!?")
    return text.lower()

def normphon(phon):
    phon = phon.strip(",.;:!?")
    return phon

def make_lexicon(text, phon):
    if phon.startswith("/") and phon.endswith("/"):
        phon = phon[1:-1]
    words = [normword(x) for x in text.split(" ")]
    phonwords = [cmudictify(normphon(x)) for x in phon.split(" ")]
    assert len(words) == len(phonwords)
    output = list(set(zip(words, phonwords)))
    return output

In [9]:
def make_ps_dict(entries):
    counts = {}
    output = []
    lex = sorted(entries)
    for entry in lex:
        count = 1
        if not entry[0] in counts:
            counts[entry[0]] = 1
        else:
            counts[entry[0]] += 1
            count = counts[entry[0]]
        if count != 1:
            subscript = f"({count})"
        else:
            subscript = ""
        output.append(f"{entry[0]}{subscript} {entry[1]}")
    return output

In [10]:
def make_fsg_transitions_from_text(text):
    words = [normword(x) for x in text.split(" ")]
    enum = [x for x in enumerate(words)]
    trans = [(x[0], x[0] + 1, 1.0, x[1]) for x in enum]
    return trans

In [11]:
import pocketsphinx


In [12]:
import tempfile
import pocketsphinx
from pocketsphinx import Decoder

seg_ps = (
    seg.set_frame_rate(16000)
       .set_channels(1)
       .set_sample_width(2)   # 16-bit
)

entries = make_ps_dict(lex)

with tempfile.NamedTemporaryFile(suffix=".dict") as dictf:
    dictf.write(("\n".join(entries) + "\n").encode("utf-8"))
    dictf.flush()

    decoder = Decoder(lm=None, dict=dictf.name)

    fsg = decoder.create_fsg("dummy", start_state, end_state, fsgt)
    decoder.add_fsg("dummy", fsg)
    decoder.activate_search("dummy")

    decoder.start_utt()
    decoder.process_raw(seg_ps.raw_data, full_utt=True)  # <-- raw PCM, not WAV
    decoder.end_utt()

segs = decoder.seg()
segs


NameError: name 'seg' is not defined

In [ ]:
from __future__ import annotations

import tempfile
from dataclasses import dataclass
from typing import Iterable, List, Sequence, Tuple, Optional

import pocketsphinx
from pocketsphinx import Decoder


@dataclass(frozen=True)
class WordSeg:
    word: str
    start_s: float
    end_s: float
    prob: float  # decoder posterior-ish for that segment (as provided)


def _ensure_ps_audio(seg):
    """
    PocketSphinx expects mono, 16kHz, 16-bit little-endian PCM.
    pydub gives us raw_data in that format once we coerce it.
    """
    return seg.set_frame_rate(16000).set_channels(1).set_sample_width(2)


def _make_linear_fsg_transitions(words: Sequence[str]) -> Tuple[int, int, List[Tuple[int, int, float, str]]]:
    """
    Build a single-path FSG: word0 -> word1 -> ... -> wordN-1
    """
    trans = [(i, i + 1, 1.0, w) for i, w in enumerate(words)]
    start_state = 0
    end_state = len(words)
    return start_state, end_state, trans


def forced_align_fsg(
    seg,  # pydub.AudioSegment
    dict_entries: Sequence[str],
    words: Optional[Sequence[str]] = None,
    fsg_transitions: Optional[Sequence[Tuple[int, int, float, str]]] = None,
    start_state: Optional[int] = None,
    end_state: Optional[int] = None,
    hmm: Optional[str] = None,
    logfn: Optional[str] = None,
) -> List[WordSeg]:
    """
    Forced-align `seg` with a custom pronunciation dictionary and an FSG.

    Provide either:
      - `words` (we build a linear FSG), OR
      - (`fsg_transitions`, `start_state`, `end_state`) if you already built them.

    `dict_entries` is a list of lines like "word(2) W ER D".
    """
    if (words is None) == (fsg_transitions is None):
        raise ValueError("Provide either `words` OR `fsg_transitions` (with start_state/end_state).")

    if fsg_transitions is None:
        start_state, end_state, fsg_transitions = _make_linear_fsg_transitions(words)  # type: ignore[arg-type]
    else:
        if start_state is None or end_state is None:
            raise ValueError("If providing fsg_transitions, you must also provide start_state and end_state.")

    seg_ps = _ensure_ps_audio(seg)

    # Write dictionary to a temp file (PocketSphinx expects a path)
    with tempfile.NamedTemporaryFile(suffix=".dict") as dictf:
        dict_blob = ("\n".join(dict_entries) + "\n").encode("utf-8")
        dictf.write(dict_blob)
        dictf.flush()

        # Build decoder config
        # - lm=None because we use an FSG
        # - dict=dictf.name to use our custom pronunciations
        # - optionally hmm=... to pin the acoustic model
        decoder_kwargs = {"lm": None, "dict": dictf.name}
        if hmm is not None:
            decoder_kwargs["hmm"] = hmm
        if logfn is not None:
            decoder_kwargs["logfn"] = logfn

        decoder = Decoder(**decoder_kwargs)

        fsg = decoder.create_fsg("utt", start_state, end_state, list(fsg_transitions))
        decoder.add_fsg("utt", fsg)
        decoder.activate_search("utt")

        decoder.start_utt()
        decoder.process_raw(seg_ps.raw_data, full_utt=True)
        decoder.end_utt()

        out: List[WordSeg] = []
        for s in decoder.seg():
            # s.word, s.start_frame, s.end_frame, s.prob are the usual fields
            # Frame rate in PocketSphinx is 100 fps by default (0.01s/frame).
            start_s = float(s.start_frame) * 0.01
            end_s = float(s.end_frame) * 0.01
            out.append(WordSeg(word=s.word, start_s=start_s, end_s=end_s, prob=float(s.prob)))

        return out
